# 🚀 ComfyUI 启动服务


在 Google Colab 上启动 ComfyUI 服务，模型持久化存储到 Google Drive。


## 📋 运行前准备
- ✅ 运行时类型设为 **GPU**（Runtime → Change runtime type → GPU）
- ✅ （可选）已运行 `comfyui_colab_with_manager.ipynb` 安装 Manager


## ▶️ 运行顺序


| 步骤 | 单元格 | 说明 |
|------|--------|------|
| 1 | 初始化 | 挂载 Drive + 克隆 ComfyUI + 创建符号链接 + 安装依赖 |
| 2 | Cloudflared | 下载隧道工具 |
| 3 | 启动服务 | 后台启动 ComfyUI |
| 4 | 启动隧道 | 获取公网访问地址 |
| 5 | 查看日志 | (可选) 排查问题 |


## 📂 架构说明

```
/content/ComfyUI/                    ← 临时（每次克隆）
├── main.py                          ← 程序入口
├── models/     → symlink → Drive    ← 持久化
├── input/      → symlink → Drive    ← 持久化
├── output/     → symlink → Drive    ← 持久化
├── custom_nodes/ → symlink → Drive  ← 持久化（含 Manager）
└── user/       → symlink → Drive    ← 持久化
```


## 📦 相关文件


| 文件 | 功能 |
|------|------|
| **comfyui_colab_with_manager.ipynb** | 安装 Manager + 自定义节点依赖 |
| **modelsDownload.ipynb** | 独立下载模型到 Drive |


## ❓ 常见问题


- **隧道超时**: 检查 ComfyUI 日志 (`/content/comfyui.log`)
- **模型不显示**: 确保符号链接指向 Drive（运行步骤 1）

In [ ]:
# ============================================
# 步骤 1: 初始化环境（一体化启动）
# ============================================
# 本单元格执行：
# 1. 挂载 Google Drive
# 2. 克隆 ComfyUI 到本地临时目录 /content/ComfyUI
# 3. 创建符号链接指向 Drive（models/input/output/custom_nodes/user）
# 4. 安装 PyTorch 等依赖
#
# ⚠️ 绝不删除 Drive 中的任何内容！
# ============================================

import sys
import os
import shutil
import subprocess
from pathlib import Path

print("=" * 70)
print("🚀 ComfyUI 初始化（程序临时 + 数据持久化）")
print("=" * 70)

# ============================================
# 1. 挂载 Google Drive
# ============================================
print("\n📂 [1/5] 挂载 Google Drive...")

from google.colab import drive
drive.mount('/content/drive', force_remount=False)

DRIVE_ROOT = Path('/content/drive/MyDrive')
if not DRIVE_ROOT.exists():
    raise SystemExit("❌ Drive 未挂载成功，请重新授权后重试")

DRIVE_COMFY_BASE = DRIVE_ROOT / 'ComfyUI'
DRIVE_COMFY_BASE.mkdir(parents=True, exist_ok=True)
print(f"✓ Drive 数据目录: {DRIVE_COMFY_BASE}")

# ============================================
# 2. 克隆 ComfyUI 到本地临时目录
# ============================================
print("\n📦 [2/5] 克隆 ComfyUI 到本地...")

LOCAL_COMFY = Path('/content/ComfyUI')

# 清理旧的本地目录（临时目录，可安全删除）
if LOCAL_COMFY.exists():
    print(f"  清理旧的本地目录: {LOCAL_COMFY}")
    shutil.rmtree(LOCAL_COMFY, ignore_errors=True)

# 克隆最新版本
os.chdir('/content')
result = subprocess.run(
    ['git', 'clone', '--depth=1', 'https://github.com/comfyanonymous/ComfyUI'],
    capture_output=True, text=True
)
if result.returncode != 0:
    print(f"✗ 克隆失败: {result.stderr}")
    raise SystemExit("❌ ComfyUI 克隆失败，请检查网络")

if not (LOCAL_COMFY / 'main.py').exists():
    raise SystemExit("❌ 克隆后未找到 main.py，请重试")

print(f"✓ ComfyUI 已克隆到: {LOCAL_COMFY}")

# ============================================
# 3. 创建符号链接（本地 → Drive）
# ============================================
print("\n🔗 [3/5] 创建符号链接...")

# 需要链接的目录
DIRS_TO_LINK = ['models', 'input', 'output', 'custom_nodes', 'user']

for dir_name in DIRS_TO_LINK:
    local_path = LOCAL_COMFY / dir_name
    drive_path = DRIVE_COMFY_BASE / dir_name
    
    # 确保 Drive 目标目录存在（不删除，只创建）
    drive_path.mkdir(parents=True, exist_ok=True)
    
    # 删除克隆产生的本地空目录（这是临时目录，可删）
    if local_path.exists() and not local_path.is_symlink():
        # 检查是否为空或只有空子目录
        try:
            shutil.rmtree(local_path)
        except Exception as e:
            print(f"  ⚠ 无法删除本地目录 {dir_name}: {e}")
            continue
    
    # 创建符号链接
    if local_path.is_symlink():
        local_path.unlink()
    
    try:
        os.symlink(str(drive_path), str(local_path))
        print(f"  ✓ {dir_name}: {local_path} → {drive_path}")
    except Exception as e:
        print(f"  ✗ {dir_name}: 链接失败 - {e}")

# ============================================
# 4. 创建模型子目录（确保 Drive 结构完整）
# ============================================
print("\n📂 [4/5] 初始化模型目录结构...")

model_subdirs = [
    'checkpoints', 'vae', 'clip', 'clip_vision', 'controlnet', 
    't2i_adapter', 'loras', 'embeddings', 'upscale_models', 
    'gligen', 'ipadapter', 'text_encoders', 'diffusion_models',
    'image_encoders', 'audio_encoders', 'unet', 'configs', 'taesd'
]
for subdir in model_subdirs:
    (DRIVE_COMFY_BASE / 'models' / subdir).mkdir(parents=True, exist_ok=True)
print(f"  ✓ 已确保 {len(model_subdirs)} 个模型子目录存在")

# ============================================
# 5. 安装依赖
# ============================================
print("\n📦 [5/5] 安装 Python 依赖...")

# 核心依赖
!pip install -q accelerate einops transformers>=4.28.1 safetensors>=0.4.2 aiohttp pyyaml Pillow scipy tqdm psutil tokenizers>=0.13.3
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121
!pip install -q torchsde kornia>=0.7.1 spandrel soundfile sentencepiece
# 视频处理依赖（ComfyUI 新版本必需）
!pip install -q av

print("✓ 依赖安装完成")

# ============================================
# 生成 extra_model_paths.yaml
# ============================================
extra_yaml_content = """# ComfyUI 模型路径配置（自动生成）
comfyui:
  base_path: /content/drive/MyDrive/ComfyUI/
  is_default: true
  checkpoints: models/checkpoints/
  vae: models/vae/
  clip: models/clip/
  clip_vision: models/clip_vision/
  controlnet: models/controlnet/
  t2i_adapter: models/t2i_adapter/
  loras: models/loras/
  embeddings: models/embeddings/
  upscale_models: models/upscale_models/
  gligen: models/gligen/
  ipadapter: models/ipadapter/
  text_encoders: models/text_encoders/
  diffusion_models: models/diffusion_models/
  unet: models/unet/
  configs: models/configs/
  custom_nodes: custom_nodes/
"""

(LOCAL_COMFY / 'extra_model_paths.yaml').write_text(extra_yaml_content)
print("✓ 已生成 extra_model_paths.yaml")

# ============================================
# 完成
# ============================================
print("\n" + "=" * 70)
print("✅ 初始化完成！")
print("=" * 70)
print(f"\n程序目录: {LOCAL_COMFY} (临时)")
print(f"数据目录: {DRIVE_COMFY_BASE} (持久化)")
print(f"\nPython 版本: {sys.version.split()[0]}")

# 导出变量供后续单元格使用
comfy_found = LOCAL_COMFY
COMFYUI_PORT = 8188

In [ ]:
# ============================================
# 步骤 2: 下载并安装 Cloudflared (Cloudflare 隧道工具)
# ============================================

import subprocess
import os
import shutil
import urllib.request
import time
import uuid

print("准备安装 Cloudflared...")

# 已安装则复用
existing = shutil.which("cloudflared")
if existing:
    print(f"✓ 已检测到 Cloudflared: {existing}")
    result = subprocess.run([existing, "--version"], capture_output=True, text=True)
    print(f"  版本: {result.stdout.strip() or result.stderr.strip()}")
    cloudflared_path = existing
else:
    # 为避免 ETXTBSY（文本文件忙），每次写入使用唯一的临时文件名
    base_dir = "/content"
    tmp_name = f"cloudflared-{os.getpid()}-{int(time.time())}-{uuid.uuid4().hex[:6]}"
    tmp_path = os.path.join(base_dir, tmp_name)
    cloudflared_path = tmp_path

    # 方案 A: 通过 apt 安装（优先）
    try:
        print("尝试通过 apt 安装 cloudflared...")
        subprocess.run(["apt-get", "update"], check=True)
        # 同时确保 curl 和 wget 可用
        subprocess.run(["apt-get", "install", "-y", "cloudflared", "curl", "wget"], check=True)
        existing = shutil.which("cloudflared")
        if existing:
            print(f"✓ apt 安装成功: {existing}")
            cloudflared_path = existing
        else:
            print("⚠ apt 安装后未找到可执行文件，继续回退下载")
    except Exception as e:
        print(f"⚠ apt 安装失败: {e}")

    # 方案 B: 直接下载二进制（curl/wget/urllib 回退）到唯一的临时文件
    if not shutil.which("cloudflared"):
        urls = [
            "https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64",
            "https://github.com/cloudflare/cloudflared/releases/download/2024.12.0/cloudflared-linux-amd64",
            "https://github.com/cloudflare/cloudflared/releases/download/2024.11.0/cloudflared-linux-amd64",
        ]
        downloaded = False
        for url in urls:
            # 每次尝试都生成新的临时文件名，规避之前失败导致的 busy 状态
            tmp_name = f"cloudflared-{os.getpid()}-{int(time.time())}-{uuid.uuid4().hex[:6]}"
            tmp_path = os.path.join(base_dir, tmp_name)
            try:
                print(f"尝试使用 curl 下载: {url}")
                subprocess.run(["curl", "-L", "-o", tmp_path, url], check=True)
                cloudflared_path = tmp_path
                downloaded = True
                break
            except Exception as e:
                print(f"curl 下载失败: {e}")
                try:
                    print("尝试使用 wget 下载...")
                    subprocess.run(["wget", "-q", "-O", tmp_path, url], check=True)
                    cloudflared_path = tmp_path
                    downloaded = True
                    break
                except Exception as e2:
                    print(f"wget 下载失败: {e2}")
                    try:
                        print("尝试使用 urllib.request 下载...")
                        req = urllib.request.Request(url, headers={"User-Agent": "Mozilla/5.0"})
                        with urllib.request.urlopen(req) as resp, open(tmp_path, "wb") as f:
                            f.write(resp.read())
                        cloudflared_path = tmp_path
                        downloaded = True
                        break
                    except Exception as e3:
                        print(f"urllib 下载失败: {e3}")
        
        if downloaded:
            os.chmod(cloudflared_path, 0o755)
            print(f"✓ Cloudflared 已下载到: {cloudflared_path}")
        else:
            raise RuntimeError("✗ 无法获取 Cloudflared 二进制，请检查网络或稍后重试")

    # 打印版本信息（若是下载的临时文件，直接执行）
    result = subprocess.run([cloudflared_path, "--version"], capture_output=True, text=True)
    print(f"  版本: {result.stdout.strip() or result.stderr.strip()}")

In [ ]:
# ============================================
# 步骤 3: 后台启动 ComfyUI 服务器
# ============================================

import subprocess
import time
import os
import sys
from pathlib import Path

# ComfyUI 服务器端口
COMFYUI_PORT = 8188

# 使用步骤 1 中设置的路径
if 'comfy_found' in globals() and comfy_found:
    comfy_dir = Path(str(comfy_found))
else:
    comfy_dir = Path('/content/ComfyUI')

if not comfy_dir.exists() or not (comfy_dir / 'main.py').exists():
    print("✗ 未找到 ComfyUI 目录，请先运行步骤 1（初始化）")
    print(f"  检查路径: {comfy_dir}")
else:
    print("正在启动 ComfyUI 服务器...")
    print(f"使用目录: {comfy_dir}")

    # 后台启动 ComfyUI
    comfyui_process = subprocess.Popen(
        [sys.executable, "main.py", "--listen", "127.0.0.1", "--port", str(COMFYUI_PORT)],
        cwd=str(comfy_dir),
        stdout=open("/content/comfyui.log", "w"),
        stderr=subprocess.STDOUT,
        start_new_session=True
    )

    print(f"✓ ComfyUI 服务器已在后台启动!")
    print(f"  PID: {comfyui_process.pid}")
    print(f"  本地地址: http://127.0.0.1:{COMFYUI_PORT}")
    print(f"  日志文件: /content/comfyui.log")

    # 等待服务器启动
    print("\n等待服务器启动...")
    time.sleep(10)
    print("✓ 服务器应该已经准备就绪!")

In [ ]:
# ============================================
# 步骤 4: 启动 Cloudflare 隧道并获取公网访问地址
# ============================================

import subprocess
import re
import time
from threading import Thread
from queue import Queue
import shutil
import os

COMFYUI_PORT = 8188

# 自动检测 cloudflared 路径
cloudflared_path = shutil.which("cloudflared")
if not cloudflared_path:
    # 查找 /content 下所有 cloudflared-* 文件
    candidates = [os.path.join("/content", f) for f in os.listdir("/content") if f.startswith("cloudflared-")]
    if candidates:
        cloudflared_path = max(candidates, key=lambda p: os.path.getmtime(p))

if not cloudflared_path or not os.path.exists(cloudflared_path):
    print("✗ 未找到 Cloudflared，请先运行步骤 2（安装 Cloudflared）")
else:
    tunnel_url_queue = Queue()

    def start_tunnel():
        process = subprocess.Popen(
            [cloudflared_path, "tunnel", "--url", f"http://127.0.0.1:{COMFYUI_PORT}"],
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )
        for line in iter(process.stdout.readline, ''):
            print(line.strip())
            match = re.search(r'(https://[a-zA-Z0-9-]+\.trycloudflare\.com)', line)
            if match:
                tunnel_url_queue.put(match.group(1))
                break
        process.wait()

    print("正在启动 Cloudflare 隧道...")
    print("=" * 50)
    tunnel_thread = Thread(target=start_tunnel, daemon=True)
    tunnel_thread.start()

    try:
        tunnel_url = tunnel_url_queue.get(timeout=30)
        print("\n" + "=" * 50)
        print("🎉 Cloudflare 隧道已建立!")
        print("=" * 50)
        print(f"\n🌐 公网访问地址: {tunnel_url}")
        print("\n提示:")
        print("  - 请在浏览器中打开上述地址访问 ComfyUI")
        print("  - 此地址为临时地址，每次运行会变化")
        print("  - 隧道会持续运行直到 Colab 实例关闭")
        print("=" * 50)
    except:
        print("⚠ 获取隧道 URL 超时，请检查上方日志")

In [ ]:
# ============================================
# [可选] 辅助工具: 查看 ComfyUI 运行日志
# ============================================

# 查看最近的 ComfyUI 日志
!tail -n 50 /content/comfyui.log